In [113]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib as plt


In [114]:
df = pd.read_csv(r"C:\Users\rymot\Desktop\Data Cleaning\data\dirty_cafe_sales.csv")

In [115]:
#Display the first 10 rows of the Df
head = df.head(10)
df.head(10)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,NaN,2023-03-31
6,TXN_4433211,UNKNOWN,3,3.0,9.0,ERROR,Takeaway,2023-10-06
7,TXN_6699534,Sandwich,4,4.0,16.0,Cash,UNKNOWN,2023-10-28
8,TXN_4717867,NaN,5,3.0,15.0,NaN,Takeaway,2023-07-28
9,TXN_2064365,Sandwich,5,4.0,20.0,NaN,In-store,2023-12-31


In [116]:
#Display the columns
df.columns

Index(['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total Spent',
       'Payment Method', 'Location', 'Transaction Date'],
      dtype='object')

In [117]:
print(df.describe())

       Transaction ID   Item Quantity Price Per Unit Total Spent  \
count           10000   9667     9862           9821        9827   
unique          10000     10        7              8          19   
top       TXN_1961373  Juice        5            3.0         6.0   
freq                1   1171     2013           2429         979   

        Payment Method  Location Transaction Date  
count             7421      6735             9841  
unique               5         4              367  
top     Digital Wallet  Takeaway          UNKNOWN  
freq              2291      3022              159  


In [118]:
print("Data types:")
print(df.dtypes)

print("\nMissing values:")
print(df.isnull().sum())

print("\nUnique values per column:")
print(df.nunique())


Data types:
Transaction ID      object
Item                object
Quantity            object
Price Per Unit      object
Total Spent         object
Payment Method      object
Location            object
Transaction Date    object
dtype: object

Missing values:
Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

Unique values per column:
Transaction ID      10000
Item                   10
Quantity                7
Price Per Unit          8
Total Spent            19
Payment Method          5
Location                4
Transaction Date      367
dtype: int64


In [119]:
df = df.replace('', np.nan)                   

In [120]:
#Check for null values
missing_summary = pd.DataFrame({
    'null_count': df.isnull().sum(),
    'null_percentage': (df.isnull().sum() / len(df)) * 100,
})
missing_summary

,null_count,null_percentage
Transaction ID,0,0.00
Item,333,3.33
Quantity,138,1.38
Price Per Unit,179,1.79
Total Spent,173,1.73
Payment Method,2579,25.79
Location,3265,32.65
Transaction Date,159,1.59


In [121]:
error_counts = df.apply(lambda col: col.astype(str).str.lower().str.count('error').sum())
unknown_counts = df.apply(lambda col: col.astype(str).str.lower().str.count('unknown').sum())

error_unknown_summary = pd.DataFrame({
    'error_count': error_counts,
    'unknown_count': unknown_counts
})

error_unknown_summary

,error_count,unknown_count
Transaction ID,0,0
Item,292,344
Quantity,170,171
Price Per Unit,190,164
Total Spent,164,165
Payment Method,306,293
Location,358,338
Transaction Date,142,159


In [122]:
df.head(10)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,NaN,2023-03-31
6,TXN_4433211,UNKNOWN,3,3.0,9.0,ERROR,Takeaway,2023-10-06
7,TXN_6699534,Sandwich,4,4.0,16.0,Cash,UNKNOWN,2023-10-28
8,TXN_4717867,NaN,5,3.0,15.0,NaN,Takeaway,2023-07-28
9,TXN_2064365,Sandwich,5,4.0,20.0,NaN,In-store,2023-12-31


In [ ]:
# Convert relevant columns to numeric, coercing errors to NaN
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['Price Per Unit'] = pd.to_numeric(df['Price Per Unit'], errors='coerce')
df['Total Spent'] = pd.to_numeric(df['Total Spent'], errors='coerce')

In [124]:
print(df[['Quantity', 'Price Per Unit', 'Total Spent']].dtypes)
# Should now show float64 or int64 instead of object

Quantity          float64
Price Per Unit    float64
Total Spent       float64
dtype: object


In [102]:
nan_count = df["Quantity"].isna().sum()
print(f"Number of NaN values in 'Quantity': {nan_count}")


Number of NaN values in 'Quantity': 479


In [127]:
# Fill missing "Total Spent" where Price & Quantity are available
df.loc[df["Total Spent"].isna() & ~df["Price Per Unit"].isna() & ~df["Quantity"].isna(), 
       "Total Spent"] = df["Price Per Unit"] * df["Quantity"]

# Fill missing "Price Per Unit" where Total Spent & Quantity are available
df.loc[df["Price Per Unit"].isna() & ~df["Total Spent"].isna() & ~df["Quantity"].isna(), 
       "Price Per Unit"] = df["Total Spent"] / df["Quantity"]

# Fill missing "Quantity" where Total Spent & Price Per Unit are available
df.loc[df["Quantity"].isna() & ~df["Total Spent"].isna() & ~df["Price Per Unit"].isna(), 
       "Quantity"] = df["Total Spent"] / df["Price Per Unit"]

In [ ]:
#df = df.dropna(subset=["Price Per Unit", "Quantity", "Total Spent"])

In [128]:
#Looking for outliers

numeric_cols = df.select_dtypes(include=[np.number]).columns
df_z = pd.DataFrame()

for col in numeric_cols:
    df_z[f"{col}_zscore"] = np.abs(stats.zscore(df[col], nan_policy='omit'))  # Omits NaN values
df = df.reset_index(drop=True)  # Reset index to avoid alignment issues
outliers = df[(df_z > 3).any(axis=1)] 
print(f"Found {len(outliers)} outliers.")

Found 0 outliers.
